In [1]:
% pylab inline
import os
import time

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave, imread, imresize

import theano
import theano.tensor as T
from keras.applications import vgg16
from keras.preprocessing.image import load_img, img_to_array
from project_ulti import *

Populating the interactive namespace from numpy and matplotlib


Using Theano backend.


In [2]:
# set image paths
root_dir = os.path.abspath('.')

base_image_path = os.path.join(root_dir, '1-content.jpg')
ref_image_path = os.path.join(root_dir, '4-style.jpg')

In [3]:
## set variables

# set image size
img_nrows = 400
img_ncols = 400

# set image weights
style_weight = 1.
content_weight = 0.0005

In [4]:
base_img = preprocess_image(base_image_path)
ref_img = preprocess_image(ref_image_path)
base_image=T.tensor4('base_image')
base_image=base_img
ref_image=T.tensor4('reference_image')
ref_image=ref_img
final_image= T.tensor4('final_image')


In [5]:

input_tensor = T.concatenate([base_image, ref_image, final_image], axis=0)

In [6]:
model = vgg16.VGG16(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('VGG-16 from imagenet')

GG-16 from imagenet


In [7]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [8]:
def content_loss(base, final):
    return T.sum(T.square(final - base))


    
def style_loss(style, final):
    S = gram_matrix(style)
    F = gram_matrix(final)
    channels = 3
    size = img_nrows * img_ncols
    return T.sum(T.square(S - F)) / (4. * (channels ** 2) * (size ** 2))



In [9]:

loss =theano.shared(numpy.cast[theano.config.floatX](0.), name='loss')
layer_features = outputs_dict['block4_conv2']
base_image_features = layer_features[0, :, :, :]
final_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features, final_features)

In [10]:

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_features = layer_features[1, :, :, :]
    final_features = layer_features[2, :, :, :]
    sl = style_loss(style_features, final_features)
    loss += (style_weight / len(feature_layers)) * sl


In [11]:
style=[]
LF1 = outputs_dict['block1_conv1']
LF2 = outputs_dict['block2_conv1']
LF3 = outputs_dict['block3_conv1']
LF4 = outputs_dict['block4_conv1']
style.append(LF1[0])
style.append(LF2[0])
style.append(LF3[0])
style.append(LF4[0])
style.append(LF1[1])
style.append(LF2[1])
style.append(LF3[1])
style.append(LF4[1])
F_outputs = theano.function([final_image], style,allow_input_downcast=True,on_unused_input='ignore')

def eval_feature(x):
    x = x.reshape((1, 3, img_nrows, img_ncols))
    outs = F_outputs(x)    #x should be final_image
    style = outs     #outs[0] is loss value     
    return style   #return [loss_value,numeric gradie

In [12]:
# gradients
grads = T.grad(loss, final_image)

In [13]:
outputs = [loss]
outputs.append(grads)
f_outputs = theano.function([final_image], outputs,allow_input_downcast=True)

In [14]:
def eval_loss_and_grads(x):
    x = x.reshape((1, 3, img_nrows, img_ncols))
    outs = f_outputs(x)
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [15]:
'''
This part conclude eval_loss_and_grads function and Evaluator class. It is a very common helper tool
to show the loss and gradient value. Many papers has mention this code. And we can also find the code
from famous Github: https://github.com/alexjc/neural-doodle. From alexjc the author of the orignal paper 
'Semantic Style Transfer and Turning Two-Bit Doodles into Fine Artwork. Many thanks to Alex J. Champandard
'''
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        style = eval_feature(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        
        self.style=style
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
    def FFF(self,x):
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return self.style 

evaluator = Evaluator()

In [16]:
# take input image
x = preprocess_image(base_image_path)

In [17]:
# iterate and optimize
for i in range(2):
    print('Start of iteration', i)
    
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    every_layer_feature=evaluator.FFF(x)
    print('Current loss value:', min_val)
    # save current generated image
    if i == 4:
        img = deprocess_image(x.copy())
        fname = '_at_iteration_%d_.png' % i
        imsave(fname, img)
        end_time = time.time()
        print('Image saved as', fname)

('Start of iteration', 0)
('Current loss value:', array(1424080512.0, dtype=float32))
('Start of iteration', 1)
('Current loss value:', array(286484064.0, dtype=float32))
